# Pre-processing News Data

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [2]:
INPUT = 'data/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [3]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339


In [4]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['data/news_vnexpress\\khoa-hoc\\00133.txt']
[4]
['Mời độc giả đặt câu hỏi tại đây\n']

Tổng số  văn bản: 1339


## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [12]:
# load dữ liệu các stopwords 

#---> Code ở đây
with open('data/vietnamese-stopwords.txt', 'r', encoding = 'utf-8') as f:
    stopwords = f.readlines()
stopwords = [word.strip() for word in stopwords]
print(f"Số lượng stopwords: {len(stopwords)}")
print(stopwords[:10])
# Chuyển hoá dữ liệu text về dạng vector TF 
#     - loại bỏ từ dừng
#     - sinh từ điển
module_count_vector = CountVectorizer(stop_words=stopwords) 
#Chuyển đổi dữ liệu thủ công
X_counts = module_count_vector.fit_transform(data_train.data) #chuyển văn bản về dạng ma trận đếm tần số xuất hiện của từ
tfidf_transformer = TfidfTransformer() 

# Chuyển đổi dữ liệu tự động với Pipeline
# tfidf_transformer = Pipeline([('vect', module_count_vector),
#                     ('tfidf', TfidfTransformer()),
#                     ])


# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 

data_preprocessed = tfidf_transformer.fit_transform(X_counts) #chuyển ma trận đếm tần số về dạng tf-idf

X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")

Số lượng stopwords: 1942
['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy']


d:\ANACONDA\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bao', 'bay', 'biến', 'biệt', 'bây', 'bõm', 'bảo', 'bất', 'bẩy', 'bập', 'bắt', 'bội', 'chao', 'chi', 'chia', 'chu', 'chui', 'chuẩn', 'chà', 'chành', 'chí', 'chót', 'chùn', 'chăn', 'chũn', 'chưng', 'chạnh', 'chả', 'chầm', 'chầy', 'chập', 'chắn', 'chẳng', 'chết', 'chốc', 'chừ', 'chừng', 'coi', 'cu', 'cá', 'câu', 'cóc', 'công', 'cạnh', 'cảm', 'cầu', 'cật', 'cắt', 'cổ', 'cụ', 'cục', 'cực', 'da', 'dà', 'dĩ', 'dưng', 'dần', 'dầu', 'dịp', 'dở', 'dụng', 'gian', 'giá', 'giác', 'giời', 'ha', 'hiện', 'hoàn', 'hèn', 'hình', 'hô', 'hầu', 'hậu', 'hẳn', 'hồ', 'hỗ', 'hội', 'hợp', 'hự', 'khói', 'khô', 'khăn', 'khắc', 'khẳng', 'kia', 'kiện', 'kê', 'kì', 'kìa', 'kỳ', 'lai', 'le', 'liên', 'liệt', 'loạt', 'luận', 'luật', 'luốt', 'lình', 'lí', 'lô', 'lý', 'lập', 'lẽ', 'lị', 'lự', 'lực', 'mày', 'mòi', 'mù', 'mạng', 'mấy', '


Số lượng từ trong từ điển: 12797
Kích thước dữ liệu sau khi xử lý: (1339, 12797)
Kích thước nhãn tương ứng: (1339,)


In [13]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.14048828 0.        ]]
5


In [14]:
sum(sum(X[100].toarray() != 0))

np.int64(289)

In [15]:
print(X[100])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 289 stored elements and shape (1, 12797)>
  Coords	Values
  (0, 81)	0.015211595534715633
  (0, 100)	0.020769547555053954
  (0, 156)	0.023262873797037953
  (0, 188)	0.04722498744523733
  (0, 269)	0.03324036700472501
  (0, 392)	0.024783841259467605
  (0, 397)	0.034232954192352914
  (0, 418)	0.048918066523847725
  (0, 662)	0.022769929356223215
  (0, 909)	0.033182248640039956
  (0, 1194)	0.05117031195708949
  (0, 1209)	0.10234062391417897
  (0, 1219)	0.05117031195708949
  (0, 1271)	0.016005705134410072
  (0, 1590)	0.034926623903062066
  (0, 1631)	0.023493949966261335
  (0, 1783)	0.04734509560871461
  (0, 1866)	0.05117031195708949
  (0, 2076)	0.014547704347804936
  (0, 2101)	0.02693672471116765
  (0, 2111)	0.02577563465433512
  (0, 2135)	0.04322051581452055
  (0, 2140)	0.01566196368628259
  (0, 2159)	0.01608450478874651
  (0, 2170)	0.02950839772591026
  :	:
  (0, 12272)	0.021259537682082122
  (0, 12454)	0.0758997005019029
  (0, 1